In [1]:
import pickle
import pandas as pd
import os
from statsmodels.tsa.api import VAR

In [2]:
# with open('var_golden_model.pkl', 'rb') as f:
#     data = pickle.load(f)
    
# print(data)



In [3]:
# my_object = {'name': 'John', 'age': 30}

# with open('my_object.pickle', 'wb') as file:
#     pickle.dump(my_object, file)

In [4]:
# with open('my_object.pickle', 'rb') as file:
#     loaded_object = pickle.load(file)

# print(loaded_object)

{'name': 'John', 'age': 30}


In [5]:
with open('var_golden_model_reduced.pkl', 'rb') as file:
    loaded_object = pickle.load(file)

print(loaded_object)

In [12]:
def load_expert_models(expert_path):
    files = os.listdir(expert_path)
    pickle_files = [file for file in files if file.endswith('.pkl')]
    models = {}

    for file in pickle_files:
        with open(file, 'rb') as f:
            models[file.split('.')[0]] = pickle.load(f)

    return models

expert_path = 'expert_models'
my_experts = load_expert_models(expert_path)
print(my_experts)

{'var_ctrl_stuckat0_perm_reduced': <statsmodels.tsa.vector_ar.var_model.VARResultsWrapper object at 0x0000021D9A9BC310>, 'var_ctrl_stuckat1_perm_reduced': <statsmodels.tsa.vector_ar.var_model.VARResultsWrapper object at 0x0000021D99488F70>, 'var_ctrl_valueFlip_perm_reduced': <statsmodels.tsa.vector_ar.var_model.VARResultsWrapper object at 0x0000021D9A7BD9F0>, 'var_golden_model_reduced': <statsmodels.tsa.vector_ar.var_model.VARResultsWrapper object at 0x0000021D9A7BF850>}
